In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
sq = """ 

WITH
  CICBaseTable AS ----To create combine CIC raw data combining Granted and Non Granted table
  ( -- Query FOR dfgranted
  SELECT
    digitalLoanAccountId,
    crifApplicationId,
    customerId,
    processEngineGuid,
    requestGuid,
    ContractHistoryType,
    CBContractCode,
    ContractEndDate,
    ContractPhase,
    ContractPhaseDesc,
    ContractStartDate,
    ContractStatus,
    ContractStatusDesc,
    ContractType,
    ContractTypeDesc,
    Currency,
    CurrencyDesc,
    LastUpdateDate,
    OriginalCurrency,
    OriginalCurrencyDesc,
    ProviderCodeEncrypted,
    ProviderContractNo,
    ReferenceNo,
    Role,
    RoleDesc,
    BilledAmount,
    BoardResolutionFlag,
    BoardResolutionFlagDesc,
    CancellationDate,
    CardReferenceCode,
    ChargedAmount,
    CreditLimit,
    CreditPurpose,
    CreditPurposeDesc,
    FinancedAmount,
    FirstPaymentDate,
    FlagCardUsed,
    HolderLiability,
    HolderLiabilityDesc,
    InstallmentType,
    InstallmentTypeDesc,
    InstallmentsNumber,
    LastChargeDate,
    LastPaymentAmount,
    LastPaymentDate,
    MinPaymentIndicator,
    MinPaymentIndicatorDesc,
    MinPaymentPercentage,
    MonthlyPaymentAmount,
    NextPayment,
    NextPaymentDate,
    OutstandingBalance,
    OutstandingBalanceUnbilled,
    OutstandingPaymentsNumber,
    OverallCreditLimit,
    OverdueDays,
    OverdueDaysDesc,
    OverduePaymentsAmount,
    OverduePaymentsNumber,
    PaymentMethod,
    PaymentMethodDesc,
    PaymentPeriodicity,
    PaymentPeriodicityDesc,
    PremiumCard,
    PremiumCardDesc,
    ReorganizedCreditCode,
    ReorganizedCreditCodeDesc,
    ServicesLinesNo,
    TimesCardUsed,
    TransactionType,
    TransactionTypeDesc,
    Utilization,
    LinkedSubject_CBSubjectCode,
    LinkedSubject_Name,
    LinkedSubject_Role,
    LinkedSubject_RoleDesc,
    Note_TypeDesc,
    Note_Text,
    Note_Type,
    run_date,
    NULL AS ContractRequestDate,
    'granted' AS SOURCE
  FROM
    prj-prod-dataplatform.risk_credit_cic_data.granted_contracts
  UNION ALL
    -- Query FOR dfnongranted
  SELECT
    digitalLoanAccountId,
    crifApplicationId,
    customerId,
    processEngineGuid,
    requestGuid,
    NULL AS ContractHistoryType,
    CBContractCode,
    NULL AS ContractEndDate,
    ContractPhase,
    ContractPhaseDesc,
    NULL AS ContractStartDate,
    NULL AS ContractStatus,
    NULL AS ContractStatusDesc,
    ContractType,
    ContractTypeDesc,
    NULL AS Currency,
    NULL AS CurrencyDesc,
    LastUpdateDate,
    NULL AS OriginalCurrency,
    NULL AS OriginalCurrencyDesc,
    ProviderCodeEncrypted,
    ProviderContractNo,
    ReferenceNo,
    Role,
    RoleDesc,
    NULL AS BilledAmount,
    NULL AS BoardResolutionFlag,
    NULL AS BoardResolutionFlagDesc,
    NULL AS CancellationDate,
    NULL AS CardReferenceCode,
    NULL AS ChargedAmount,
    CreditLimit,
    NULL AS CreditPurpose,
    NULL AS CreditPurposeDesc,
    FinancedAmount,
    NULL AS FirstPaymentDate,
    NULL AS FlagCardUsed,
    NULL AS HolderLiability,
    NULL AS HolderLiabilityDesc,
    NULL AS InstallmentType,
    NULL AS InstallmentTypeDesc,
    InstallmentsNumber,
    NULL AS LastChargeDate,
    NULL AS LastPaymentAmount,
    NULL AS LastPaymentDate,
    NULL AS MinPaymentIndicator,
    NULL AS MinPaymentIndicatorDesc,
    NULL AS MinPaymentPercentage,
    MonthlyPaymentAmount,
    NULL AS NextPayment,
    NULL AS NextPaymentDate,
    NULL AS OutstandingBalance,
    NULL AS OutstandingBalanceUnbilled,
    NULL AS OutstandingPaymentsNumber,
    NULL AS OverallCreditLimit,
    NULL AS OverdueDays,
    NULL AS OverdueDaysDesc,
    NULL AS OverduePaymentsAmount,
    NULL AS OverduePaymentsNumber,
    NULL AS PaymentMethod,
    NULL AS PaymentMethodDesc,
    PaymentPeriodicity,
    PaymentPeriodicityDesc,
    NULL AS PremiumCard,
    NULL AS PremiumCardDesc,
    NULL AS ReorganizedCreditCode,
    NULL AS ReorganizedCreditCodeDesc,
    NULL AS ServicesLinesNo,
    NULL AS TimesCardUsed,
    NULL AS TransactionType,
    NULL AS TransactionTypeDesc,
    NULL AS Utilization,
    LinkedSubject_CBSubjectCode,
    LinkedSubject_Name,
    LinkedSubject_Role,
    LinkedSubject_RoleDesc,
    Note_TypeDesc,
    Note_Text,
    Note_Type,
    run_date,
    ContractRequestDate,
    'nongranted' AS SOURCE
  FROM
    prj-prod-dataplatform.risk_credit_cic_data.notgranted_contracts )
    -- select * from CICBaseTable where digitalLoanAccountId = 'c7948327-e6a6-46ee-96fc-66c1b3b56f93'
-- SELECT  ContractHistoryType, RoleDesc, count(digitalLoanAccountId) cnt FROM  CICBaseTable where COALESCE(ContractHistoryType, 'NA') in ('Installments', 'CreditCards', 'NA') group by 1,2 order by 3 desc;
,
employementdata as
(SELECT distinct
  digitalLoanAccountId, 
  crifApplicationId, 
  customerId,   
  AnnualMonthlyIndicator, 
  Currency, 
  DateHiredFrom, 
  DateHiredTo, 
  GrossIncome,
  CAST(
    CASE 
      WHEN COALESCE(AnnualMonthlyIndicator, 'NA') LIKE 'M' THEN CAST(COALESCE(GrossIncome, '0') AS NUMERIC)
      WHEN COALESCE(AnnualMonthlyIndicator, 'NA') LIKE 'Y' THEN ROUND(CAST(COALESCE(GrossIncome, '0') AS NUMERIC)/12, 0)
      ELSE 0 
    END AS INT64
  ) AS MonthlyIncome,
  CAST(
    CASE 
      WHEN COALESCE(AnnualMonthlyIndicator, 'NA') LIKE 'M' THEN ROUND(CAST(COALESCE(GrossIncome, '0') AS NUMERIC)*12, 0)
      WHEN COALESCE(AnnualMonthlyIndicator, 'NA') LIKE 'Y' THEN CAST(COALESCE(GrossIncome, '0') AS NUMERIC)
      ELSE 0 
    END AS INT64
  ) AS AnnualIncome,    
  OccupationDesc,
  OccupationStatusDesc,
  PSIC, 
  REGEXP_REPLACE(PSICDesc, r'^\d+\s*-\s*', '') AS PSICDesc ,
  row_number() over (partition by digitalLoanAccountId order by digitalLoanAccountId ) as rnk
FROM prj-prod-dataplatform.risk_credit_cic_data.employment_data),
CICBase2Table as 
(SELECT digitalLoanAccountId, crifApplicationId, customerId,
       processEngineGuid, requestGuid, ContractHistoryType,
       CBContractCode, ContractEndDate, ContractPhase,
       ContractPhaseDesc, ContractStartDate, ContractStatus,
       ContractStatusDesc, ContractType, ContractTypeDesc,
       Currency, CurrencyDesc, LastUpdateDate, OriginalCurrency,
       OriginalCurrencyDesc, ProviderCodeEncrypted,
       ProviderContractNo, ReferenceNo, Role, RoleDesc,
       BilledAmount, BoardResolutionFlag, BoardResolutionFlagDesc,
       CancellationDate, CardReferenceCode, ChargedAmount,
       CreditLimit, CreditPurpose, CreditPurposeDesc,
       FinancedAmount, FirstPaymentDate, FlagCardUsed,
       HolderLiability, HolderLiabilityDesc, InstallmentType,
       InstallmentTypeDesc, InstallmentsNumber, LastChargeDate,
       LastPaymentAmount, LastPaymentDate, MinPaymentIndicator,
       MinPaymentIndicatorDesc, MinPaymentPercentage,
       MonthlyPaymentAmount, NextPayment, NextPaymentDate,
       OutstandingBalance, OutstandingBalanceUnbilled,
       OutstandingPaymentsNumber, OverallCreditLimit, OverdueDays,
       OverdueDaysDesc, OverduePaymentsAmount,
       OverduePaymentsNumber, PaymentMethod, PaymentMethodDesc,
       PaymentPeriodicity, PaymentPeriodicityDesc, PremiumCard,
       PremiumCardDesc, ReorganizedCreditCode,
       ReorganizedCreditCodeDesc, ServicesLinesNo, TimesCardUsed,
       TransactionType, TransactionTypeDesc, Utilization,
       LinkedSubject_CBSubjectCode, LinkedSubject_Name,
       LinkedSubject_Role, LinkedSubject_RoleDesc, Note_TypeDesc,
       Note_Text, Note_Type, run_date, ContractRequestDate,  SOURCE
,
  CASE
    WHEN ContractPhaseDesc = 'Active' AND ContractStatusDesc = '' THEN 'Neutral'
    WHEN ContractPhaseDesc = 'Active' AND ContractStatusDesc is null THEN 'Neutral'
    WHEN ContractPhaseDesc = 'Active' AND ContractStatusDesc = 'Pre-Activated' THEN 'Good'
    WHEN ContractPhaseDesc = 'Active' AND ContractStatusDesc = 'Foreclosure' THEN 'Good'
    WHEN ContractPhaseDesc = 'Closed' AND ContractStatusDesc = '' THEN 'Good'
    WHEN ContractPhaseDesc = 'Closed' AND ContractStatusDesc is null THEN 'Good'
    WHEN ContractPhaseDesc = 'Closed in advance' AND ContractStatusDesc = '' THEN 'Good'
    WHEN ContractPhaseDesc = 'Closed in advance' AND ContractStatusDesc is null THEN 'Good'
    WHEN ContractPhaseDesc = 'Closed in advance' AND ContractStatusDesc = 'Foreclosure' THEN 'Good'
    WHEN ContractStatusDesc IN ('Debt Assumption', 'Repossessed') THEN 'Neutral'
    WHEN ContractStatusDesc IN (
      'Write-off (BLW)', 'Past Due', 'Blocked by the Bank due to Credit Reasons',
      'Under dispute / non performing', 'Under litigation / Delinquent',
      'Blocked or Closed voluntary by the Customer', 'Blocked or Closed due to Restructuring',
      'There are unpaid amounts, Negotiated Settlement', 'Previous delinquency settled',
      'Write-off and Credit transferred to third party / Collection',
      'Write-off and Fully Settled', 'Blocked by the Bank due to card lost/stolen',
      'Blocked by the Bank due to fraud', 'Dispute / Litigation contested'
    ) THEN 'Bad'
    ELSE 'Unknown'
  END AS Repaymentcategory,
CASE
    WHEN ContractTypeDesc IN ('Salary loan', 'Personal Loan', 'Unsecured loan', 'Vehicle Loan', 'Mortgage/Real Estate', 'Time Loan', 'Short Term Loan', 'Benefit Loan', 'Home equity loan', 'Agricultural Loan', 'Student Loan', 'Vehicle leasing', 'Credit Card', 'Credit Card - Shared Limit', 'Credit Card - MultiCurrency', 'Revolving Credit', 'Trust Loan', 'Credit Line') 
      OR (ContractTypeDesc = 'Term Loan' AND CreditPurposeDesc NOT LIKE 'Small and Medium Enterprise Loans%')
      OR (ContractTypeDesc = 'Loan Line' AND CreditPurposeDesc NOT LIKE 'Small and Medium Enterprise Loans%')
      OR (CreditPurposeDesc LIKE 'Loans to Individual%' AND ContractTypeDesc != 'Business Loan')
      OR (CreditPurposeDesc LIKE 'Microfinance Loans' AND ContractTypeDesc != 'Business Loan')
      OR (CreditPurposeDesc LIKE 'Other Agricultural Credit' AND ContractTypeDesc != 'Business Loan')
      OR (ContractHistoryType LIKE 'Installments' AND ContractTypeDesc = 'Term Loan' and CreditPurposeDesc is null)
      OR (ContractHistoryType is null AND ContractTypeDesc = 'Term Loan' and CreditPurposeDesc is null)
      OR CreditPurposeDesc IN ('Agrarian Reform', 'Development Loan Incentives - Socialized Low Cost Housing (Loans to individuals for housing purposes )')
      OR ContractHistoryType = 'CreditCards'
    THEN 'B2C'
    
    WHEN ContractTypeDesc IN ('Business Loan', 'Real estate leasing', 'Equipment leasing')
      OR CreditPurposeDesc IN ('Development Loan Incentives - Cooperatives', 'Development Loan Incentives - Educational Inst.', 'Loan to Government - GOCCs (Other Financial)', 'Loan to Government - GOCCs (Social Security Institutions)', 'Loan to Government - LGUs', 'Loan to Government - National Government', 'Loans to Private Corporation (Financial)', 'Loans to Private Corporation (Non-Financial)', 'Small and Medium Enterprise Loans (Medium Scale Enterprise)', 'Small and Medium Enterprise Loans (Small Scale Enterprise)')
      OR (ContractTypeDesc = 'Vehicle Loan' AND CreditPurposeDesc NOT LIKE 'Loans to Individual%')
      OR (ContractTypeDesc = 'Loan Line' AND CreditPurposeDesc LIKE 'Small and Medium Enterprise Loans%')
      OR (ContractTypeDesc = 'Term Loan' AND CreditPurposeDesc LIKE 'Small and Medium Enterprise Loans%')
    THEN 'B2B'
    
    ELSE 'Unknown'
  END AS BusinessType,
 CASE
    WHEN ContractTypeDesc = 'Time Loan' THEN 'Time Loans'
    WHEN ContractTypeDesc IN ('Short Term Loan', 'Term Loan') THEN 'Short and Term Loans'
    WHEN ContractTypeDesc = 'Home equity loan' THEN 'Home Equity Loans'
    WHEN ContractTypeDesc IN ('Credit Card', 'Credit Card - MultiCurrency', 'Credit Card - Shared Limit') THEN 'Credit Cards'
    WHEN ContractTypeDesc IN ('Loan Line', 'Credit Line') THEN 'Credit Lines'
    WHEN ContractTypeDesc IN ('Mortgage/Real Estate', 'Real estate leasing') THEN 'Real Estate Loans'
    WHEN ContractTypeDesc = 'Trust Loan' THEN 'Trust Loans'
    WHEN ContractTypeDesc = 'Personal Loan' THEN 'Personal Loans'
    ELSE 'Other Loans'
  END AS loan_segment
from CICBaseTable 
  where COALESCE(ContractHistoryType, 'NA') in ('Installments', 'CreditCards', 'NA')
  and COALESCE(RoleDesc, 'NA') in ('Borrower', 'Co-Borrower', 'NA')
),
CICBase3Table as
(select distinct * FROM  CICBase2Table where BusinessType in ('B2C', 'Unknown')
)
-- select distinct * from CICBase3Table where (digitalLoanAccountid||crifApplicationId||run_date||CBContractCode) = 'c7948327-e6a6-46ee-96fc-66c1b3b56f93taran-2629742-80860572024-07-05302970230';
,
custtname as (SELECT distinct  cast(custId as numeric) custid, firstName, middleName, LastName FROM `prj-prod-dataplatform.dl_loans_db_raw.tdbk_loan_customer_details` 
),
stepAtablebase as
(
select 
(a.digitalLoanAccountid||b.crifApplicationId||b.run_date||b.CBContractCode) uniquekey,
a.digitalLoanAccountId,
a.customerId, cn.Firstname, cn.middleName, cn.LastName,
a.loanAccountNumber,
a.flagDisbursement,
a.disbursementDateTime,
a.termsAndConditionsSubmitDateTime,
a.natureofwork,
a.subIndustryDescription,
a.industryDescription,
case when a.reloan_flag = 1 and a.loantype not like 'FLEXUP'then 'Reloan'
      when a.loantype = 'FLEXUP' and a.new_loan_type = 'Flex-up' and a.reloan_flag = 0 and a.flagDisbursement = 1 then 'Flex-up' 
              else a.new_loan_type end as LoanProduct, b.crifApplicationId, 
       processEngineGuid, requestGuid, ContractHistoryType,
       CBContractCode, ContractEndDate, ContractPhase,
       ContractPhaseDesc, ContractStartDate, ContractStatus,
       ContractStatusDesc, ContractType, ContractTypeDesc,
       b.Currency, CurrencyDesc, LastUpdateDate, OriginalCurrency,
       OriginalCurrencyDesc, ProviderCodeEncrypted,
       ProviderContractNo, ReferenceNo, Role, RoleDesc,
       BilledAmount, BoardResolutionFlag, BoardResolutionFlagDesc,
       CancellationDate, CardReferenceCode, ChargedAmount,
       CreditLimit, CreditPurpose, CreditPurposeDesc,
       FinancedAmount, FirstPaymentDate, FlagCardUsed,
       HolderLiability, HolderLiabilityDesc, InstallmentType,
       InstallmentTypeDesc, InstallmentsNumber, LastChargeDate,
       LastPaymentAmount, LastPaymentDate, MinPaymentIndicator,
       MinPaymentIndicatorDesc, MinPaymentPercentage,
       MonthlyPaymentAmount, NextPayment, NextPaymentDate,
       b.OutstandingBalance, OutstandingBalanceUnbilled,
       OutstandingPaymentsNumber, OverallCreditLimit, OverdueDays,
       OverdueDaysDesc, OverduePaymentsAmount,
       OverduePaymentsNumber, PaymentMethod, PaymentMethodDesc,
       PaymentPeriodicity, PaymentPeriodicityDesc, PremiumCard,
       PremiumCardDesc, ReorganizedCreditCode,
       ReorganizedCreditCodeDesc, ServicesLinesNo, TimesCardUsed,
       TransactionType, TransactionTypeDesc, Utilization,
       LinkedSubject_CBSubjectCode, LinkedSubject_Name,
       LinkedSubject_Role, LinkedSubject_RoleDesc, Note_TypeDesc,
       Note_Text, Note_Type, run_date, ContractRequestDate,  SOURCE, Repaymentcategory, BusinessType, loan_segment
       , ed.AnnualMonthlyIndicator, ed.Currency, ed.DateHiredFrom, ed.DateHiredTo, ed.GrossIncome, ed.MonthlyIncome, ed.AnnualIncome, ed.OccupationDesc, ed.OccupationStatusDesc, ed.PSIC, ed.PSICDesc
FROM `risk_credit_mis.loan_master_table` a 
inner join CICBase3Table b
ON a.digitalLoanAccountId = b.digitalLoanAccountId
    AND a.crifApplicationId = b.crifApplicationId
left join (select * from employementdata where rnk = 1) ed on ed.digitalLoanAccountId = a.digitalLoanAccountId
Left join custtname cn on cn.custid = a.customerId
where a.disbursementDateTime is not null
and date_trunc(a.disbursementDateTime, day) >= '2023-01-10'
and date_trunc(a.disbursementDateTime, day) < current_date()
),
stepAtable2base as 
(select *, row_number() over(partition by uniquekey order by uniquekey) rnk from stepAtablebase)
,
-- select * from stepAtable2base where rnk > 1;
-- select * from stepAtablebase where  (digitalLoanAccountid||crifApplicationId||run_date||CBContractCode) = 'c7948327-e6a6-46ee-96fc-66c1b3b56f93taran-2629742-80860572024-07-05302970230'
-- select (digitalLoanAccountid||crifApplicationId||run_date||CBContractCode), count((digitalLoanAccountid||crifApplicationId||run_date||CBContractCode)) from stepAtable2base group by 1 having count((digitalLoanAccountid||crifApplicationId||run_date||CBContractCode)) > 1;
base as 
(select a.*, d.obsFSPD30, d.defFSPD30 
, case when date_trunc(a.disbursementDateTime, day) <= '2024-03-31' then 'Train_Validation' 
         when date_trunc(a.disbursementDateTime, day) >'2024-03-31' and date_trunc(a.disbursementDateTime, day) <= '2024-06-07' then 'Test' else 'Other' end targetdataselectiontype
from stepAtable2base a 
inner join 
(
    SELECT  
        loanAccountNumber
        , SUM(CASE WHEN obs_min_inst_def30 >= 1 THEN 1 ELSE 0 END) as obsFPD30
        , SUM(CASE WHEN min_inst_def30 = 1 THEN 1 else 0 END) as defFPD30
        , sum(case when obs_min_inst_def30>=1 then (select max(disbursedloanamount) from `risk_credit_mis.loan_master_table` where loanAccountNumber = a1.loanAccountNumber) else 0 end) as obs_fpd30_vol
        , sum(case when min_inst_def30=1 then (select max(disbursedloanamount) from `risk_credit_mis.loan_master_table` where loanAccountNumber = a1.loanAccountNumber) else 0 end) as def_fpd30_vol
        , SUM(CASE WHEN obs_min_inst_def30 >= 2 THEN 1 ELSE 0 END) as obsFSPD30
        , SUM(CASE WHEN obs_min_inst_def30 >= 2 AND (min_inst_def30 = 2 or min_inst_def30 = 1) THEN 1 else 0 END) as defFSPD30
        , SUM(CASE WHEN obs_min_inst_def30 >= 2 THEN (select max(disbursedloanamount) from `risk_credit_mis.loan_master_table` where loanAccountNumber = a1.loanAccountNumber) ELSE 0 END) as obsFSPD30_vol
        , SUM(CASE WHEN obs_min_inst_def30 >= 2 AND (min_inst_def30 = 2 or min_inst_def30 = 1) THEN (select max(disbursedloanamount) from `risk_credit_mis.loan_master_table` where loanAccountNumber = a1.loanAccountNumber) else 0 END) as defFSPD30_vol
        , SUM(CASE WHEN obs_min_inst_def30 >= 3 THEN 1 ELSE 0 END) as obsFSTPD30
        , SUM(CASE WHEN obs_min_inst_def30 >= 3 AND (min_inst_def30 = 3 or min_inst_def30 = 2 or min_inst_def30 = 1) THEN 1 else 0 END) as defFSTPD30
        , SUM(CASE WHEN obs_min_inst_def30 >= 3 THEN (select max(disbursedloanamount) from `risk_credit_mis.loan_master_table` where loanAccountNumber = a1.loanAccountNumber) ELSE 0 END) as obsFSTPD30_vol
        , SUM(CASE WHEN obs_min_inst_def30 >= 3 AND (min_inst_def30 = 3 or min_inst_def30 = 2 or min_inst_def30 = 1) THEN (select max(disbursedloanamount) from `risk_credit_mis.loan_master_table` where loanAccountNumber = a1.loanAccountNumber) else 0 END) as defFSTPD30_vol
    FROM `risk_credit_mis.loan_deliquency_data` a1 
    GROUP BY 1
  ) d
ON a.loanAccountNumber = d.loanAccountNumber
where 
a.rnk = 1
and a.LoanProduct in ('Quick', 'SIL-Instore')
)
select * from base 
--  where digitalLoanAccountId = '42c268b9-1fe7-445c-b459-8c66d0483884'
-- select digitalLoanAccountId, count(digitalLoanAccountId) cnt from base group by 1 having count(digitalLoanAccountId) > 1
-- select uniquekey, count(uniquekey) from base group by 1 having count(uniquekey) > 1
-- and 
-- uniquekey = 'c7948327-e6a6-46ee-96fc-66c1b3b56f93taran-2629742-80860572024-07-05302970230'
;
"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
df.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\CIC Data Analysis\CICModel\DataPreparation\Data\cicfinaldataset20240807_v3.csv", index = False)
print(f"The rows and columns in cic final dataset are:\t {df.shape}")
pd.set_option("display.max_columns", None)
df.head(10)


Job ID 1a14c00f-1d37-4caf-9b27-035ac2386387 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in cic final dataset are:	 (245037, 112)


,uniquekey,digitalLoanAccountId,customerId,Firstname,middleName,LastName,loanAccountNumber,flagDisbursement,disbursementDateTime,termsAndConditionsSubmitDateTime,natureofwork,subIndustryDescription,industryDescription,LoanProduct,crifApplicationId,processEngineGuid,requestGuid,ContractHistoryType,CBContractCode,ContractEndDate,ContractPhase,ContractPhaseDesc,ContractStartDate,ContractStatus,ContractStatusDesc,ContractType,ContractTypeDesc,Currency,CurrencyDesc,LastUpdateDate,OriginalCurrency,OriginalCurrencyDesc,ProviderCodeEncrypted,ProviderContractNo,ReferenceNo,Role,RoleDesc,BilledAmount,BoardResolutionFlag,BoardResolutionFlagDesc,CancellationDate,CardReferenceCode,ChargedAmount,CreditLimit,CreditPurpose,CreditPurposeDesc,FinancedAmount,FirstPaymentDate,FlagCardUsed,HolderLiability,HolderLiabilityDesc,InstallmentType,InstallmentTypeDesc,InstallmentsNumber,LastChargeDate,LastPaymentAmount,LastPaymentDate,MinPaymentIndicator,MinPaymentIndicatorDesc,MinPaymentPercentage,MonthlyPaymentAmount,NextPayment,NextPaymentDate,OutstandingBalance,OutstandingBalanceUnbilled,OutstandingPaymentsNumber,OverallCreditLimit,OverdueDays,OverdueDaysDesc,OverduePaymentsAmount,OverduePaymentsNumber,PaymentMethod,PaymentMethodDesc,PaymentPeriodicity,PaymentPeriodicityDesc,PremiumCard,PremiumCardDesc,ReorganizedCreditCode,ReorganizedCreditCodeDesc,ServicesLinesNo,TimesCardUsed,TransactionType,TransactionTypeDesc,Utilization,LinkedSubject_CBSubjectCode,LinkedSubject_Name,LinkedSubject_Role,LinkedSubject_RoleDesc,Note_TypeDesc,Note_Text,Note_Type,run_date,ContractRequestDate,SOURCE,Repaymentcategory,BusinessType,loan_segment,AnnualMonthlyIndicator,Currency_1,DateHiredFrom,DateHiredTo,GrossIncome,MonthlyIncome,AnnualIncome,OccupationDesc,OccupationStatusDesc,PSIC,PSICDesc,rnk,obsFSPD30,defFSPD30,targetdataselectiontype
0,000d5b99-ffd3-45ad-b650-32fe43a95dc7taran-2614...,000d5b99-ffd3-45ad-b650-32fe43a95dc7,2614354,NENITA,UMALI,CAIDIC,60826143540016,1,2024-06-27 16:59:48,2024-06-27 16:55:44,None,None,None,SIL-Instore,taran-2614354-9950345,9be21cf75100ec9fe03b3f154c42f1fe0bfc7364,16b6e2be-fd2d-4b5f-87cb-01e72ffbb5ab,CreditCards,O04057747,None,AC,Active,2018-06-23,None,None,31,Credit Card,PHP,Philippine peso,2022-07-31,PHP,Philippine peso,CC005,None,1,B,Borrower,None,None,None,None,None,15446,167000,None,None,None,None,None,None,None,V,variable,None,2022-05-11,103177,2022-07-29,None,None,None,None,850,2022-08-01,8655,7167,None,None,0,Paid as agreed / Current,0,0,CCR,Credit card payment,M,monthly installments-30 days,None,None,0,Credit is not re-organized,None,0,PCC,PRIMARY CREDIT CARD,None,None,None,None,None,None,None,None,2024-06-27,None,granted,Neutral,B2C,Credit Cards,Y,PHP,None,None,360000,30000,360000,None,Self Employed,None,None,1,0,0,Other
1,0010fa50-dedd-471b-93b8-7e2cabf2b71ftonik-bnpl...,0010fa50-dedd-471b-93b8-7e2cabf2b71f,2383606,EDILBERTO,GARCIA,DELA CRUZ,60823836060011,1,2024-01-29 16:06:12,2024-01-29 16:03:47,Sales/Marketing Professional,Others,Services,SIL-Instore,tonik-bnpl-437300,effb4cb0-be7c-11ee-b07b-0242ace6000f,efdd6470-be7c-11ee-82ea-0242ace60004,Installments,K02698546,2018-08-23,CL,Closed,2017-11-15,None,None,12,Personal Loan,PHP,Philippine peso,2018-08-31,PHP,Philippine peso,PF002,None,1,B,Borrower,None,0,No,None,None,None,None,None,None,4029,None,None,None,None,None,None,9,None,782,2018-08-22,None,None,None,582,0,None,0,None,0,None,0,Paid as agreed / Current,0,0,None,None,M,monthly installments-30 days,None,None,0,Credit is not re-organized,None,None,NA,NOT APPLICABLE,None,None,None,None,None,None,None,None,2024-01-29,None,granted,Good,B2C,Personal Loans,None,None,None,None,None,0,0,None,None,None,None,1,1,0,Train_Validation
2,00437977-13bc-4093-ae5d-a596cabe6c5btaran-2610...,00437977-13bc-4093-ae5d-a596cabe6c5b,2610898,BEVERLY JOY,ARCENAS,VARGAS,60826108980017,1,2024-06-26 13:19:02,2024-06-26 13:15:13,Other Non Professional Services,Hotel / Resort / Lodge,Services,SIL-Instore,taran-2610898-2682737,9be21cf75100ec9fe03b3f154c42f

In [3]:
df['disbursementDateTime'][df['obsFSPD30']==1].max()

Timestamp('2024-05-07 20:09:44')

In [4]:
import pandas as pd

# Assuming df is your original DataFrame

# Calculate duplicate counts
duplicate_counts = df.groupby('targetdataselectiontype')['digitalLoanAccountId'].apply(lambda x: x.duplicated().sum()).reset_index(name='duplicate_count')

# Count total unique digitalLoanAccountId
unique_counts = df.groupby('targetdataselectiontype')['digitalLoanAccountId'].nunique().reset_index(name='unique_count')

# Count where defFSPD30 = 1 and obsFSPD30 = 1
def_count = df[df['defFSPD30'] == 1].groupby('targetdataselectiontype')['digitalLoanAccountId'].nunique().reset_index(name='defFSPD30_count')
obs_count = df[df['obsFSPD30'] == 1].groupby('targetdataselectiontype')['digitalLoanAccountId'].nunique().reset_index(name='obsFSPD30_count')

# Sum defFSPD30 and obsFSPD30
sum_values = df.groupby('targetdataselectiontype')[['defFSPD30', 'obsFSPD30']].sum().reset_index()

# Merge all the dataframes
merged_df = pd.merge(unique_counts, duplicate_counts, on='targetdataselectiontype')
merged_df = pd.merge(merged_df, def_count, on='targetdataselectiontype', how='left')
merged_df = pd.merge(merged_df, obs_count, on='targetdataselectiontype', how='left')
merged_df = pd.merge(merged_df, sum_values, on='targetdataselectiontype', how='left')

# Fill NaN values with 0 for count columns
merged_df[['defFSPD30_count', 'obsFSPD30_count']] = merged_df[['defFSPD30_count', 'obsFSPD30_count']].fillna(0)

# Calculate ratios
merged_df['FSPD30_ratio'] = merged_df['defFSPD30'] / merged_df['obsFSPD30']
merged_df['FSPD30_percentage'] = merged_df['FSPD30_ratio'].apply(lambda x: f"{x:.2%}")

# Create the pivot table
pivot_table = pd.pivot_table(
    merged_df,
    values=['unique_count', 'duplicate_count', 'defFSPD30_count', 'obsFSPD30_count', 'defFSPD30', 'obsFSPD30', 'FSPD30_ratio'],
    index=['targetdataselectiontype'],
    aggfunc='sum',
    margins=True,
    margins_name='Total'
)

# Format the percentage column
pivot_table['FSPD30_percentage'] = pivot_table['FSPD30_ratio'].apply(lambda x: f"{x:.2%}")

# Reorder columns if desired
pivot_table = pivot_table[['unique_count', 'duplicate_count', 'defFSPD30_count', 'obsFSPD30_count', 'defFSPD30', 'obsFSPD30', 'FSPD30_ratio', 'FSPD30_percentage']]

# Display the pivot table
pivot_table

,unique_count,duplicate_count,defFSPD30_count,obsFSPD30_count,defFSPD30,obsFSPD30,FSPD30_ratio,FSPD30_percentage
targetdataselectiontype,,,,,,,,
Other,8674,40863,0.0,0.0,0,0,NaN,nan%
Test,11710,50371,583.0,5012.0,2453,25566,0.095948,9.59%
Train_Validation,25200,108219,3451.0,25200.0,17902,133419,0.134179,13.42%
Total,45584,199453,4034.0,30212.0,20355,158985,<NA>,nan%


In [5]:
sq = """
select * from prj-prod-dataplatform.risk_credit_cic_data.cic_summary
;"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns of cicsummary are:\t {dfd.shape}")

Job ID 2b1f4922-3e51-4d55-9ec0-69866f3be41a successfully executed: |          | 
Downloading: 100%|██████████|
The rows and columns of cicsummary are:	 (427136, 93)


In [6]:
# Merge both the data based on digitalLoanAccountId
dfd = dfd[['digitalLoanAccountId','DescisionValue', 'exclusionRuleLabelDescription', 'ScoreRaw', 'ScoreRange',
       'Accounts30Days', 'Accounts30DaysOther', 'Accounts60Days',
       'Accounts60DaysOther', 'Accounts90Days', 'Accounts90DaysOther',
       'AgeAtLoanTermination', 'AgeAtLoanTerminationMax',
       'CreditAvgCreditLimit', 'CreditHighestCreditLimit',
       'CreditMaxOutstandingBalance', 'CreditMaxOutstandingBalanceDate',
       'CreditMaxOverdueAmount', 'CreditMaxOverdueAmountDate',
       'CreditMaxOverdueDays', 'CreditMaxOverdueDaysDate',
       'CreditNumberOfContracts', 'CreditTotalCreditLimit',
       'CreditTotalOutstandingBalanceAmount', 'CreditTotalOverdueAmount', 'InstMaxOverdueAmount', 'InstMaxOverdueAmountDate',
       'InstMaxOverdueDays', 'InstMaxOverdueDaysDate', 'InstNumberOfContracts',
       'InstTotalMonthlyPaymentsAmount', 'InstTotalOutstandingBalanceAmount',
       'InstTotalOverdueAmount', 'LoanToValue', 'MonthsOfBooks',
       'MonthsOfBooksOther', 'NonInstAvgCreditLimit',
       'NonInstHighestCreditLimit', 'NonInstNumberOfContracts',
       'NonInstTotalCreditLimit', 'NonInstTotalOverdraftAmount',
       'NonInstTotalUtilization', 'NumberOfContract', 'NumberOfContractAC',
       'NumberOfContractACOther', 'NumberOfContractCLCA',
       'NumberOfContractCLCAOther', 'NumberOfContractOther',
       'NumberOfContractRF', 'NumberOfContractRFOther', 'NumberOfContractRN',
       'NumberOfContractRNOther', 'NumberOfContractRQ',
       'NumberOfContractRQOther', 'OverlimitFlag', 'OverlimitFlagOther',
       'ReportingProvidersNumber', 'ReportingProvidersNumberOther',
       'SubjectEventDate', 'SubjectInfoTypeCode', 'SubjectInfoTypeCodeDesc',
       'TotalOutstanding', 'TotalOutstandingOther', 'UtilityMaxBilledAmount',
       'UtilityMaxBilledAmountDate', 'UtilityMaxOutstandingBalance',
       'UtilityMaxOutstandingBalanceDate', 'UtilityMaxOverdueAmount',
       'UtilityMaxOverdueAmountDate', 'UtilityMaxOverdueDays',
       'UtilityMaxOverdueDaysDate', 'UtilityNumberOfContracts',
       'UtilityTotalBilledAmount', 'UtilityTotalOutstandingBalanceAmount',
       'UtilityTotalOverdueAmount', 'UtilizationRateCreditCard',
       'UtilizationRateCreditCardOther', 'UtilizationRateNonInstallment',
       'UtilizationRateNonInstallmentOther','scoreLabelDesc']].copy()



In [7]:
# Assuming you have a dataframe called 'df'
duplicate_rows = dfd[dfd.duplicated()]

# To see all duplicate rows, including the first occurrence:
all_duplicates = dfd[dfd.duplicated(keep=False)]

print(f"The number of duplicate rows are:\t{len(duplicate_rows)}")

The number of duplicate rows are:	68


In [8]:
# Remove duplicates, keeping the last occurrence
df_cleaned = dfd.drop_duplicates(keep='last')

print(f"Original dataframe shape: {dfd.shape}")
print(f"Cleaned dataframe shape: {df_cleaned.shape}")

Original dataframe shape: (427136, 80)
Cleaned dataframe shape: (427068, 80)


In [9]:
mergeddf = pd.merge(df, df_cleaned, left_on='digitalLoanAccountId', right_on='digitalLoanAccountId', how = 'left')
mergeddf.head()

,uniquekey,digitalLoanAccountId,customerId,Firstname,middleName,LastName,loanAccountNumber,flagDisbursement,disbursementDateTime,termsAndConditionsSubmitDateTime,natureofwork,subIndustryDescription,industryDescription,LoanProduct,crifApplicationId,processEngineGuid,requestGuid,ContractHistoryType,CBContractCode,ContractEndDate,ContractPhase,ContractPhaseDesc,ContractStartDate,ContractStatus,ContractStatusDesc,ContractType,ContractTypeDesc,Currency,CurrencyDesc,LastUpdateDate,OriginalCurrency,OriginalCurrencyDesc,ProviderCodeEncrypted,ProviderContractNo,ReferenceNo,Role,RoleDesc,BilledAmount,BoardResolutionFlag,BoardResolutionFlagDesc,CancellationDate,CardReferenceCode,ChargedAmount,CreditLimit,CreditPurpose,CreditPurposeDesc,FinancedAmount,FirstPaymentDate,FlagCardUsed,HolderLiability,HolderLiabilityDesc,InstallmentType,InstallmentTypeDesc,InstallmentsNumber,LastChargeDate,LastPaymentAmount,LastPaymentDate,MinPaymentIndicator,MinPaymentIndicatorDesc,MinPaymentPercentage,MonthlyPaymentAmount,NextPayment,NextPaymentDate,OutstandingBalance,OutstandingBalanceUnbilled,OutstandingPaymentsNumber,OverallCreditLimit,OverdueDays,OverdueDaysDesc,OverduePaymentsAmount,OverduePaymentsNumber,PaymentMethod,PaymentMethodDesc,PaymentPeriodicity,PaymentPeriodicityDesc,PremiumCard,PremiumCardDesc,ReorganizedCreditCode,ReorganizedCreditCodeDesc,ServicesLinesNo,TimesCardUsed,TransactionType,TransactionTypeDesc,Utilization,LinkedSubject_CBSubjectCode,LinkedSubject_Name,LinkedSubject_Role,LinkedSubject_RoleDesc,Note_TypeDesc,Note_Text,Note_Type,run_date,ContractRequestDate,SOURCE,Repaymentcategory,BusinessType,loan_segment,AnnualMonthlyIndicator,Currency_1,DateHiredFrom,DateHiredTo,GrossIncome,MonthlyIncome,AnnualIncome,OccupationDesc,OccupationStatusDesc,PSIC,PSICDesc,rnk,obsFSPD30,defFSPD30,targetdataselectiontype,DescisionValue,exclusionRuleLabelDescription,ScoreRaw,ScoreRange,Accounts30Days,Accounts30DaysOther,Accounts60Days,Accounts60DaysOther,Accounts90Days,Accounts90DaysOther,AgeAtLoanTermination,AgeAtLoanTerminationMax,CreditAvgCreditLimit,CreditHighestCreditLimit,CreditMaxOutstandingBalance,CreditMaxOutstandingBalanceDate,CreditMaxOverdueAmount,CreditMaxOverdueAmountDate,CreditMaxOverdueDays,CreditMaxOverdueDaysDate,CreditNumberOfContracts,CreditTotalCreditLimit,CreditTotalOutstandingBalanceAmount,CreditTotalOverdueAmount,InstMaxOverdueAmount,InstMaxOverdueAmountDate,InstMaxOverdueDays,InstMaxOverdueDaysDate,InstNumberOfContracts,InstTotalMonthlyPaymentsAmount,InstTotalOutstandingBalanceAmount,InstTotalOverdueAmount,LoanToValue,MonthsOfBooks,MonthsOfBooksOther,NonInstAvgCreditLimit,NonInstHighestCreditLimit,NonInstNumberOfContracts,NonInstTotalCreditLimit,NonInstTotalOverdraftAmount,NonInstTotalUtilization,NumberOfContract,NumberOfContractAC,NumberOfContractACOther,NumberOfContractCLCA,NumberOfContractCLCAOther,NumberOfContractOther,NumberOfContractRF,NumberOfContractRFOther,NumberOfContractRN,NumberOfContractRNOther,NumberOfContractRQ,NumberOfContractRQOther,OverlimitFlag,OverlimitFlagOther,ReportingProvidersNumber,ReportingProvidersNumberOther,SubjectEventDate,SubjectInfoTypeCode,SubjectInfoTypeCodeDesc,TotalOutstanding,TotalOutstandingOther,UtilityMaxBilledAmount,UtilityMaxBilledAmountDate,UtilityMaxOutstandingBalance,UtilityMaxOutstandingBalanceDate,UtilityMaxOverdueAmount,UtilityMaxOverdueAmountDate,UtilityMaxOverdueDays,UtilityMaxOverdueDaysDate,UtilityNumberOfContracts,UtilityTotalBilledAmount,UtilityTotalOutstandingBalanceAmount,UtilityTotalOverdueAmount,UtilizationRateCreditCard,UtilizationRateCreditCardOther,UtilizationRateNonInstallment,UtilizationRateNonInstallmentOther,scoreLabelDesc
0,000d5b99-ffd3-45ad-b650-32fe43a95dc7taran-2614...,000d5b99-ffd3-45ad-b650-32fe43a95dc7,2614354,NENITA,UMALI,CAIDIC,60826143540016,1,2024-06-27 16:59:48,2024-06-27 16:55:44,None,None,None,SIL-Instore,taran-2614354-9950345,9be21cf75100ec9fe03b3f154c42f1fe0bfc7364,16b6e2be-fd2d-4b5f-87cb-01e72ffbb5ab,CreditCards,O04057747,None,AC,Active,2018-06-23,N

In [10]:
# 1. Check for duplicate rows
duplicates = mergeddf.duplicated()

# 2. Get the number of duplicate rows
num_duplicates = duplicates.sum()

print(f"Number of duplicate rows: {num_duplicates}")

# 3. View the duplicate rows
print("\nDuplicate rows:")
print(mergeddf[duplicates])

Number of duplicate rows: 0

Duplicate rows:
Empty DataFrame
Columns: [uniquekey, digitalLoanAccountId, customerId, Firstname, middleName, LastName, loanAccountNumber, flagDisbursement, disbursementDateTime, termsAndConditionsSubmitDateTime, natureofwork, subIndustryDescription, industryDescription, LoanProduct, crifApplicationId, processEngineGuid, requestGuid, ContractHistoryType, CBContractCode, ContractEndDate, ContractPhase, ContractPhaseDesc, ContractStartDate, ContractStatus, ContractStatusDesc, ContractType, ContractTypeDesc, Currency, CurrencyDesc, LastUpdateDate, OriginalCurrency, OriginalCurrencyDesc, ProviderCodeEncrypted, ProviderContractNo, ReferenceNo, Role, RoleDesc, BilledAmount, BoardResolutionFlag, BoardResolutionFlagDesc, CancellationDate, CardReferenceCode, ChargedAmount, CreditLimit, CreditPurpose, CreditPurposeDesc, FinancedAmount, FirstPaymentDate, FlagCardUsed, HolderLiability, HolderLiabilityDesc, InstallmentType, InstallmentTypeDesc, InstallmentsNumber, LastC

In [11]:
mergeddf.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\CIC Data Analysis\CICModel\DataPreparation\Data\CICFinalDataSetWithCICSummaryData20240807_v3.csv", index = False)